# Scrum Board Data Map

## 1. About Scrum
### Scrum is a great way for teams to get started with agile.
Scrum calls for four ceremonies that bring structure to each sprint:

  * Sprint planning
  * Daily stand-up
  * Sprint demo
  * Sprint retrospective

### [참조] Agile 
* [Wiki Scrum 정의](https://ko.wikipedia.org/wiki/%EC%8A%A4%ED%81%AC%EB%9F%BC_(%EC%95%A0%EC%9E%90%EC%9D%BC_%EA%B0%9C%EB%B0%9C_%ED%94%84%EB%A1%9C%EC%84%B8%EC%8A%A4)
* [Attlasian Scrum 정의](https://ko.atlassian.com/agile/scrum/)
* [agile 소프트웨어 개발 선언](http://www.agilemanifesto.org/)

### Consideration when creating our models.
* A task board is commonly used in Scrum-style development to manage tasks in the current sprint.
* Tasks are moved from the backlog into the set of tasks to be completed.
* Various states can be used to track the progress along the way, such as “in progress”or “in testing.”
* The preceding tasks must be part of a task for it to be in a “completed” state.

## 2. Initial Project Layout
```
$ django-admin.py startproject scrum 
$ cd scrum
$ python manage.py startapp board
```

### Caution
The startapp template has evolved in recent Django versions, and the output might be different if you are using a different version of Django. admin.py was added to the template in Django 1.6 and the migrations folder was added in 1.7.

## 3. Project Settings
**the server will not maintain any client state, the contrib.session references can be removed. This will break usage of the default Django admin,** which means those references can be removed as well.

### Changes
* remove **django.contrib.admin, django.contrib.sessions, and django.contrib.messages** from the INSTALLED_APPS
* new INSTALLED_APPS include **rest_framework and rest_framework.authtoken** as well as the board app

### Install
Use PostgreSQL, Requires installing **psycopg2, the Python driver for PostgreSQL.** We’ll need to create the database as well.

```
$ pip install psycopg2
$ createdb -E UTF-8 scrum
```

### Delete admin url pattern
All of the existing patterns have been removed and replaced with a single URL for **rest_framework.authtoken.views.obtain_auth_token. This serves as the view for exchanging a username and password combination for an API token**.

In [4]:
%%writefile scrum/scrum/urls.py

from django.conf.urls import include, url
from rest_framework.authtoken.views import obtain_auth_token

urlpatterns = [
    url(r'^api/token/', obtain_auth_token, name='api-token'),
]

Overwriting scrum/scrum/urls.py


## No Django Admin?
In this application it has been removed because the API doesn’t use or need Django’s session management, which is required to use the admin. **The browsable API, which will be provided by the django-rest- framework, can serve as a simple replacement for the admin.**

You may find that you still need to maintain the Django admin due to other applications that rely on it. You can preserve the admin and simply have two sets of settings for the project



## Models

In [3]:
%%writefile scrum/board/models.py

from django.db import models
from django.utils.translation import ugettext_lazy as _

class Sprint(models.Model): 
    """Development iteration period."""
    name = models.CharField(max_length=100, blank=True, default='')
    description = models.TextField(blank=True, default='')
    end = models.DateField(unique=True)

    def __str__(self):
        return self.name or _('Sprint ending %s') % self.end

Overwriting scrum/board/models.py


We should also note that tasks will have a handful of states:
* Not Started
* In Progress
* Testing
* Done

In [5]:
%%writefile scrum/board/models.py

from django.conf import settings
from django.db import models
from django.utils.translation import ugettext_lazy as _

class Sprint(models.Model): 
    """Development iteration period."""
    name = models.CharField(max_length=100, blank=True, default='')
    description = models.TextField(blank=True, default='')
    end = models.DateField(unique=True)

    def __str__(self):
        return self.name or _('Sprint ending %s') % self.end
    
class Task(models.Model):
    """Unit of work to be done for the sprint."""
    STATUS_TODO = 1
    STATUS_IN_PROGRESS = 2
    STATUS_TESTING = 3
    STATUS_DONE = 4
    STATUS_CHOICES = (
        (STATUS_TODO, _('Not Started')),
        (STATUS_IN_PROGRESS, _('In Progress')),
        (STATUS_TESTING, _('Testing')),
        (STATUS_DONE, _('Done')),
    )
        
    name = models.CharField(max_length=100)
    description = models.TextField(blank=True, default='')
    sprint = models.ForeignKey(Sprint, blank=True, null=True)
    status = models.SmallIntegerField(choices=STATUS_CHOICES, default=STATUS_TODO)
    order = models.SmallIntegerField(default=0)
    assigned = models.ForeignKey(settings.AUTH_USER_MODEL, null=True, blank=True)
    started = models.DateField(blank=True, null=True)
    due = models.DateField(blank=True, null=True)
    completed = models.DateField(blank=True, null=True)

    def __str__(self):
        return self.name

Overwriting scrum/board/models.py


### Clear limitations in this data model
* One issue is that this tracks sprints only for a single project and assumes that all system users are involved with the project.
* The task states are also fixed in the task model, making those states unusable in the sprint model
* There is also no support here for customizing the task workflow.

These limitations are likely acceptable if the application will be used for a single project, **but obviously would not work if the intention were to build this task board as a software as a service (SaaS) product.**

### [참조] What is SaaS?
**Software as a service (or SaaS) is a way of delivering applications over the Internet—as a service**. Instead of installing and maintaining software, you simply access it via the Internet, freeing yourself from complex software and hardware management.

[Salesforce.com에서 정의한 SaaS](https://www.salesforce.com/saas/)

### Migration
1. Create model using makemigration command
model에 변경이 있을 경우, 반영한다.
```
$ python manage.py makemigrations board 
```

2. Create tables using **migrate** command
```
$ python manage.py migrate
```

3. Migrate 수행 결과
```
Operations to perform:
  Synchronize unmigrated apps: staticfiles, rest_framework
  Apply all migrations: authtoken, board, auth, contenttypes
Synchronizing apps without migrations:
  Creating tables...
    Running deferred SQL...
  Installing custom SQL...
Running migrations:
  Rendering model states... DONE
  Applying contenttypes.0001_initial... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0001_initial... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying authtoken.0001_initial... OK
  Applying board.0001_initial... OK
```

### [참조] migrate 수행 시 error 발생

#### 문제
OS X에서 migrate 명령어 수행 시, **계속 사용자 계정으로 접속을 하며, 인증에 실패했다고 오류가 발생하고 정상적으로 동작하지 않았다.**
OS X에서 사용자 계정을 생성한 적이 없으니, 당연히 인증에서 실패해서 DB에 connection이 생성되지 않는다.
```
createdb: could not connect to database template1: FATAL: password authentication failed for user
```

#### 해결방법
1. pg_hba.conf에 auto-method 값을 md5 값을 trust로 변경
  - pg_hba.conf 파일 위치 : /home/Library/PostgreSQL/[version]/data/pg_hba.conf
  - auto method 값 의미
    * md5 : 입력된 password 값을 md5 방식으로 암호화된 값으로 인증
    * trust : 암호 입력 없이 인증 처리
  - [pg_hba.conf 파일 설명](http://www.postgresql.org/docs/9.1/static/auth-pg-hba-conf.html)
  
2. OS X에 사용자 계정 생성 및 password 세팅
  * [pgAdmin SQL 툴](http://www.pgadmin.org/download/) 설치 후, 손쉽게 계정 생성 가능
  * **pgAdmin은 PostgreSQL, PPAS DB에 접속 가능한 SQL 툴이며, 무료로 사용가능한 소프트웨어**
  * **scrum/setting.py 에 DATABASE dict에 'USER', 'PASSWORD' Key에 생성한 계정 정보 값을 세팅한다.** (https://docs.djangoproject.com/en/1.8/ref/settings/)

### Create a superuser
**createsuperuser 를 통해 생성된 계정은 "auth_user" 테이블에 데이터가 등록된다.**
```
$ python manage.py createsuperuser
```

Prior to Django 1.7 there was no built-in support for migrated model schemas(**django_migration** table). Prior to Django 1.7 could create tables only using a single command called **syncdb**. 

# Designing the API
the API we want to build will look like this:
```
/api/
    /sprints/
        /<id>/
    /tasks/
        /<id>/
    /users/
        /<username>/
```

It’s **important to consider how clients will navigate this API.**

We’ve chosen not to include the version number in the URLs for the API. **While it is included in many popular APIs, we feel that the approach for versioning that aligns best with RESTful practices is to use different content types for different API versions.** However, if you really want a version number in your API, feel free to add it.

## Sprint Endpoints
**Building resources tied to Django models** with django-rest-framework is easy with **ViewSets**. To build the ViewSet for the /api/sprints/, we should **describe how the model should be serialized and deserialized by the API**.

In [1]:
%%writefile scrum/board/serializers.py

from rest_framework import serializers
from .models import Sprint

class SprintSerializer(serializers.ModelSerializer):
    class Meta:
        model = Sprint
        fields = ('id', 'name', 'description', 'end', )

Writing scrum/board/serializers.py


the **ModelViewSet** provides the **scaffolding needed for the create, read, update, delete (CRUD) operations** using the corresponding HTTP verbs.

In [3]:
%%writefile scrum/board/views.py

from rest_framework import viewsets

from .models import Sprint
from .serializers import SprintSerializer 

class SprintViewSet(viewsets.ModelViewSet):
    """API endpoint for listing and creating sprints."""

    queryset = Sprint.objects.order_by('end')
    serializer_class = SprintSerializer

Overwriting scrum/board/views.py


This view will be **explicit about its settings**. As implemented as a **mixin class**

**DefaultsMixin** will be one of the base classes for the API view classes to define these options.

In [4]:
%%writefile scrum/board/views.py
from rest_framework import authentication, permissions, viewsets

from .models import Sprint
from .serializers import SprintSerializer

class DefaultsMixin(object):
    """Default settings for view authentication, permissions, filtering and pagination."""
    
    authentication_classes = (
        authentication.BasicAuthentication,
        authentication.TokenAuthentication,
    )
    permission_classes = (
        permissions.IsAuthenticated,
    )
    paginate_by = 25
    paginate_by_param = 'page_size'
    max_paginate_by = 100
    
class SprintViewSet(DefaultsMixin, viewsets.ModelViewSet):
    """API endpoint for listing and creating sprints."""
    queryset = Sprint.objects.order_by('end')
    serializer_class = SprintSerializer

Overwriting scrum/board/views.py


### Add authentication for the user permissions.
Using **basic auth** will make it easy **to use the browsable API via a web browser.**

This example does not have fine- grained permissions, since we are working on the assumption that all users in the system are a part of the project.

**The only permission requirement is that the user is authenticated.**

## Task and User Endpoints
We need the tasks to be exposed on their own endpoint. Similar to our sprint endpoints

**status_display**
  * The status will show the number rather than the text associated with its state.
  * read-only field 

In [5]:
%%writefile scrum/board/serializers.py

from rest_framework import serializers
from .models import Sprint, Task

class SprintSerializer(serializers.ModelSerializer):
    class Meta:
        model = Sprint
        fields = ('id', 'name', 'description', 'end', )

class TaskSerializer(serializers.ModelSerializer):
    status_display = serializers.SerializerMethodField('get_status_display')
    
    class Meta: 
        model = Task
        fields = ('id', 'name', 'description', 'sprint', 
                  'status', 'status_display', 'order',
                  'assigned', 'started', 'due', 'completed', )

    def get_status_display(self, obj):
        return obj.get_status_display()

Overwriting scrum/board/serializers.py


The second issue with our **serializer is that assigned is a foreign key to the User model.**

User model might be swapped out for another and that the intent of our application is to make it as reusable as possible. 
 
We will need to use the **get_user_model** Django utility in board/serializers.py to create this switch in a clean way.

#### get_user_model()
Instead of referring to User directly, you should reference the user model using **django.contrib.auth.get_user_model()**. This method will **return the currently active User model** – the custom User model if one is specified, or User otherwise.

#### [참조] User model
* [Django User model](https://docs.djangoproject.com/en/1.8/topics/auth/customizing/#auth-custom-user)
* [get_full_name()](https://docs.djangoproject.com/en/1.8/ref/contrib/auth/#django.contrib.auth.models.User)

In [7]:
%%writefile scrum/board/serializers.py

from django.contrib.auth import get_user_model

from rest_framework import serializers

from .models import Sprint, Task

User = get_user_model()

class UserSerializer(serializers.ModelSerializer):
    full_name = serializers.CharField(source='get_full_name', read_only=True)

    class Meta:
        model = User
        fields = ('id', User.USERNAME_FIELD, 'full_name', 'is_active', )

class SprintSerializer(serializers.ModelSerializer):
    class Meta:
        model = Sprint
        fields = ('id', 'name', 'description', 'end', )

class TaskSerializer(serializers.ModelSerializer):
    assigned = serializers.SlugRelatedField(
            slug_field=User.USERNAME_FIELD, required=False)
    status_display = serializers.SerializerMethodField('get_status_display')
    
    class Meta: 
        model = Task
        fields = ('id', 'name', 'description', 'sprint', 
                  'status', 'status_display', 'order',
                  'assigned', 'started', 'due', 'completed', )

    def get_status_display(self, obj):
        return obj.get_status_display()

Overwriting scrum/board/serializers.py


### ViewSets for the tasks and users
**UserViewSet** in that it **extends from ReadOnlyModelViewSet** instead. As the name implies, this does not expose the actions to create new users or to edit existing ones through the API. 

### [참조] ViewSet
```
ViewSet <- GenericViewSet <- ModelViewSet
```

The GenericViewSet class inherits from GenericAPIView.

#### Attributes
**Basic settings:**
The following attributes control the basic view behavior.

* **queryset** - **The queryset that should be used for returning objects from this view.** Typically, you must either set this attribute, or override the get_queryset() method. If you are overriding a view method, it is important that you call get_queryset() instead of accessing this property directly, **as queryset will get evaluated once, and those results will be cached for all subsequent requests.**

* **serializer_class** - The serializer class that should be **used for validating and deserializing input, and for serializing output.** Typically, you must either set this attribute, or override the get_serializer_class() method.

* **lookup_field** - The model field that should be used to for **performing object lookup of individual model instances. Defaults to 'pk'.** Note that when using hyperlinked APIs you'll need to ensure that both the API views and the serializer classes set the lookup fields if you need to use a custom value.

* **lookup_url_kwarg** - The URL keyword argument that should be used for object lookup. **The URL conf should include a keyword argument corresponding to this value.** If unset this defaults to using the same value as lookup_field.

#### 참조 사이트
* [ViewSet 설명](http://www.django-rest-framework.org/api-guide/viewsets/#viewsets)
* [GenericAPIView 설명](http://www.django-rest-framework.org/api-guide/generic-views/)

In [16]:
%%writefile scrum/board/views.py

from django.contrib.auth import get_user_model

from rest_framework import authentication, permissions, viewsets

from .models import Sprint, Task
from .serializers import SprintSerializer, TaskSerializer, UserSerializer

User = get_user_model()

class DefaultsMixin(object):
    """Default settings for view authentication, permissions, filtering and pagination."""
    
    authentication_classes = (
        authentication.BasicAuthentication,
        authentication.TokenAuthentication,
    )
    permission_classes = (
        permissions.IsAuthenticated,
    )
    paginate_by = 25
    paginate_by_param = 'page_size'
    max_paginate_by = 100

class SprintViewSet(viewsets.ModelViewSet):
    """API endpoint for listing and creating sprints."""

    queryset = Sprint.objects.order_by('end')
    serializer_class = SprintSerializer

class TaskViewSet(DefaultsMixin, viewsets.ModelViewSet):
    """API endpoint for listing and creating tasks."""
    
    queryset = Task.objects.all()
    serializer_class = TaskSerializer
        
class UserViewSet(DefaultsMixin, viewsets.ReadOnlyModelViewSet):
    """API endpoint for listing users."""
    
    lookup_field = User.USERNAME_FIELD
    lookup_url_kwarg = User.USERNAME_FIELD
    queryset = User.objects.order_by(User.USERNAME_FIELD)
    serializer_class = UserSerializer

Overwriting scrum/board/views.py


## Connecting to the Router
**django-rest-framework** has its own URL routing extension for handling ViewSets, where each ViewSet is registered with the router for a given URL prefix.

In [11]:
%%writefile scrum/board/urls.py

from rest_framework.routers import DefaultRouter
from . import views

router = DefaultRouter()
router.register(r'sprints', views.SprintViewSet)
router.register(r'tasks', views.TaskViewSet)
router.register(r'users', views.UserViewSet)

Writing scrum/board/urls.py


Finally, this router needs to be included in the root URL configuration in scrum/urls.py.

In [12]:
%%writefile scrum/scrum/urls.py

from django.conf.urls import include, url

from rest_framework.authtoken.views import obtain_auth_token

from board.urls import router

urlpatterns = [
    url(r'^api/token/', obtain_auth_token, name='api-token'),
    url(r'^api/', include(router.urls)),
]

Overwriting scrum/scrum/urls.py


## Linking Resources
One important constraint of a RESTful application is hypermedia as the engine of application state (**HATEOAS**).

the client should be aware only of few fixed endpoints to the server. From those fixed endpoints, the client should discover the **resources available on the server through the use of descriptive resource messages.**

To give clients a uniform place to look for these **links**, each resource will have a links section in the response containing the relevant links. 
**The easiest way to start is for the resources to link back to themselves.**

* [HATEOAS 설명](http://stackoverflow.com/questions/26394303/what-does-it-mean-that-rest-should-be-hypertext-driven)

### Add Task link information
this will return the **full URI, including the hostname and protocol, along with the path.** For this, reverse needs the current request, which is passed into the serializer context by default when we’re using the standard ViewSets.

A task assigned to a sprint should point back to its sprint. You can also link from a task to its assigned user by reversing the URL if there is a user assigned

In [15]:
%%writefile scrum/board/serializers.py

from django.contrib.auth import get_user_model

from rest_framework import serializers
from rest_framework.reverse import reverse

from .models import Sprint, Task

User = get_user_model()

class SprintSerializer(serializers.ModelSerializer):
    
    links = serializers.SerializerMethodField('get_links')

    class Meta:
        model = Sprint
        fields = ('id', 'name', 'description', 'end', 'links', )

    def get_links(self, obj):
        request = self.context['request']
        return {
            'self': reverse('sprint-detail',
                kwargs={'pk': obj.pk},request=request),
        }
    
class TaskSerializer(serializers.ModelSerializer):

    assigned = serializers.SlugRelatedField(
        slug_field=User.USERNAME_FIELD, required=False)
    status_display = serializers.SerializerMethodField('get_status_display')
    links = serializers.SerializerMethodField('get_links')

    class Meta:
        model = Task
        fields = ('id', 'name', 'description', 'sprint',
            'status', 'status_display', 'order',
            'assigned', 'started', 'due', 'completed', 'links', )

    def get_status_display(self, obj):
        return obj.get_status_display()
    
    def get_links(self, obj):
        request = self.context['request']
        links = {
            'self': reverse('task-detail', 
                kwargs={'pk': obj.pk}, request=request),
            'sprint': None,
            'assigned': None
        }
        if obj.sprint_id:
            links['sprint'] = reverse('sprint-detail',
                kwargs={'pk': obj.sprint_id}, request=request)
        if obj.assigned:
            links['assigned'] = reverse('user-detail',
                kwargs={User.USERNAME_FIELD: obj.assigned}, request=request)
        return links

class UserSerializer(serializers.ModelSerializer):
    
    full_name = serializers.CharField(source='get_full_name', read_only=True)
    links = serializers.SerializerMethodField('get_links')

    class Meta:
        model = User
        fields = ('id', User.USERNAME_FIELD, 'full_name',
            'is_active', 'links', )

    def get_links(self, obj):
        request = self.context['request']
        username = obj.get_username()
        return {
            'self': reverse('user-detail',
                kwargs={User.USERNAME_FIELD: username}, request=request),
        }

Overwriting scrum/board/serializers.py


this will return the **full URI, including the hostname and protocol, along with the path.** For this, reverse needs the current request, which is passed into the serializer context by default when we’re using the standard ViewSets.

# Testing Out the API
```
$ python manage.py runserver
```

## 에러 발생
**1. Relational Field에 queryset 추가**
Taskserializer에 assigned 필드에 query set 추가한다.
'''
Relational field must provide a `queryset` argument, or set read_only=`True`.
'''

**2. serializer에 method_name 파라미터 삭제**
이대로 실행하면 Root API 정보는 확인할 수 있지만, 각 상세 링크를 클릭하면 아래와 같은 에러가 난다.

```
It is redundant to specify `get_links` on SerializerMethodField 'links' in serializer 'SprintSerializer', because it is the same as the default method name. Remove the `method_name` argument.
```

## 해결
1. User.object.all() 값을 queryset 추가
```
 assigned = serializers.SlugRelatedField(
        slug_field=User.USERNAME_FIELD, required=False,
        queryset=User.objects.all())
```
2. django rest framework version 3.X 부터는 link 메소드명을 삭제하는 것으로 변경되었다.

## 참조
* [O'REILLY 에러](http://www.oreilly.com/catalog/errata.csp?isbn=0636920032502)